<a href="https://colab.research.google.com/github/zkc1031/BadmintonNetTouch/blob/main/main_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Googleドライブ連携
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#必要ライブラリのインストール


# 感嘆符(!)をつけると、ターミナルコマンドとして実行される
!pip install "numpy<2.0" mediapipe==0.10.9 ultralytics torch
# protobufとtensorflowは、Colabにプリインストールされている互換性のあるバージョンをそのまま使うのが安定します

In [4]:
#フレームラベル付けシステム

# ライブラリのインポート
import os
import shutil
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# --- 設定 ---
# ラベル付け候補の画像が入っているフォルダ
SOURCE_DIR = '/content/drive/MyDrive/Badminton_Research/datasets/frames_to_label/'

# 分類後の画像を保存する親フォルダ
DEST_DIR_BASE = '/content/drive/MyDrive/Badminton_Research/datasets/labeled_data/'

# クラス名と、対応するフォルダ名
CLASSES = ['net_touch', 'over_net', 'normal']

# --- 関数の定義 ---

# 各クラスの保存先フォルダを作成する
for class_name in CLASSES:
    os.makedirs(os.path.join(DEST_DIR_BASE, class_name), exist_ok=True)
# スキップした画像用のフォルダも作成
os.makedirs(os.path.join(DEST_DIR_BASE, 'skipped'), exist_ok=True)

# SOURCE_DIRが存在するかチェック
if not os.path.exists(SOURCE_DIR):
    print(f"Error: Source directory not found at {SOURCE_DIR}")
    print("Please check if the path is correct and the directory exists in your Google Drive.")
else:
    # ファイルリストを取得し、シャッフルする
    image_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(image_files)

    # 現在のインデックスを管理する
    current_index = 0

    # 画像表示用のウィジェット
    image_widget = widgets.Image(format='png', width=800)

    # ボタンウィジェットの作成
    buttons = []
    for label in CLASSES + ['skip']:
        button = widgets.Button(description=label)
        buttons.append(button)

    # ボタンがクリックされたときの処理を定義
    def on_button_clicked(b):
        global current_index

        # 古い画像とボタンをクリア
        clear_output(wait=True)

        # 選択されたラベルに基づいてファイルを移動
        label = b.description
        if current_index < len(image_files):
            img_name = image_files[current_index]
            source_path = os.path.join(SOURCE_DIR, img_name)
            dest_path = os.path.join(DEST_DIR_BASE, label, img_name)

            try:
                shutil.move(source_path, dest_path)
                print(f"Moved '{img_name}' to '{label}' folder.")
            except Exception as e:
                print(f"Error moving file: {e}")

        # 次の画像へ
        current_index += 1
        show_next_image()

    # 各ボタンにクリックイベントを登録
    for button in buttons:
        button.on_click(on_button_clicked)

    # 次の画像を表示する関数
    def show_next_image():
        global current_index
        if current_index < len(image_files):
            img_name = image_files[current_index]
            img_path = os.path.join(SOURCE_DIR, img_name)

            # 画像を読み込んで表示
            with open(img_path, 'rb') as f:
                image_widget.value = f.read()

            # 進捗を表示
            progress_label = widgets.Label(f"Image {current_index + 1} / {len(image_files)}: {img_name}")

            # 画像とボタンをまとめて表示
            display(progress_label, image_widget, widgets.HBox(buttons))
        else:
            print("全ての画像の分類が完了しました！お疲れ様でした。")

    # 最初の画像を表示して開始
    if image_files:
        show_next_image()
    else:
        print("Source directory is empty or does not contain supported image files (.png, .jpg, .jpeg).")

Error: Source directory not found at /content/drive/MyDrive/Badminton_Research/datasets/frames_to_label/
Please check if the path is correct and the directory exists in your Google Drive.


In [ ]:
# 最初の画像を表示してラベリングを開始
show_next_image()